<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# Introduction à la librairie  [SparkML](https://spark.apache.org/docs/latest/ml-guide.html) (ou MLlib  DataFrame-based API) de  <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> 

Depuis Spark 2.0 la librairie MLlib, qui maniupule éxclusivement des RDD est maintenant en maintenance. Elle peut toujours être utilisé, mais il n'y aura pas de nouveaux outils développé pour cette librairie.

La principale librairie de machine learning pour spark est maintenant SparkML. SparkML est une librairie qui manipule exclusivement des DataFrame, plus facile d'utilisation que les RDD et qui permet l'utilistation d'autres services spark tel que Spark Datasources, SQL queries etc...
Si MLlib est aujourd'hui plus complète, SparkML devrait posséder exactement les même fonctions que MLlib dans la version 2.3 de Spark (Aujourd'hui la version 2.2 est disponible). La librairie MLlib, qui manipule exclusivement des RDD n'existera plus dans Spark 3.0


SparkML n'est pas le nom définitif que portera la librairie de machine learning de spark. C'est un nom temporaire créer pour déveloper un outil basé sur les DataFrame. MLlib sera le nom définitif de cette librairie.

**Résumé**: Ce tutoriel continue l'initiation à [Spark](https://spark.apache.org/) à l'aide de commandes en Python en utilisant l'API  [`PySpark`](http://spark.apache.org/docs/latest/api/python/). Dans ce calepein Nous allons manier majoritairement des DataFrame telles que celles décrite dans le calepin précédent

##  Statistique élémentaire avec <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> et  [SparkML](https://spark.apache.org/docs/latest/ml-guide.html)

La plupart des fonctions de statistiques élémentaires de <I>MLlib</I>, décrites dans le calepin 2 de cette introduction à pyspark ne sont pas encore disponible dans la librairie <I>SparkML</I>. Seul la fonction de correlation et le test d'hypothèse du χ² sont disponible. 

In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### Correlation

La fonction <I> pyspark.ml.stat.Correlation</I> permet de calculer la correlation entre chaque colonne d'une DataFrame. Les correlations disponible sont celles de <I>Pearson</I> et <I>Spearman</I>. 

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]


df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

Pearson correlation matrix:
DenseMatrix([[ 1.        ,  0.05564149,         nan,  0.40047142],
             [ 0.05564149,  1.        ,         nan,  0.91359586],
             [        nan,         nan,  1.        ,         nan],
             [ 0.40047142,  0.91359586,         nan,  1.        ]])
Spearman correlation matrix:
DenseMatrix([[ 1.        ,  0.10540926,         nan,  0.4       ],
             [ 0.10540926,  1.        ,         nan,  0.9486833 ],
             [        nan,         nan,  1.        ,         nan],
             [ 0.4       ,  0.9486833 ,         nan,  1.        ]])


### Test d'Hypothèse

Spark.ml supporte actuellement le test du Chi-2 de Pearson. Ce test permet d'effectuer un test d'indépendance pour chaque <I>features</I> vis à vis du <I>label</I>. 

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.687289278791,0.682270330336]
degreesOfFreedom: [2, 3]
statistics: [0.75,1.5]


## ML Pipeline

La librairie <I>SparkML</I>, contrairement à <I>MLlib</I>, est basé sur la notion de <B>ML Pipeline</B>. 

Un <B>ML Pipeline</B> permet de combiner différentes étapes de traitement, allant du nettoyage des données jusqu'a l'étape d'apprentissage en un seul objet appelé <I>pipeline<\I> ou <I>workflow.<\I>. 

### Estimator, Transformer, and Param

La construction d'un <B>ML Pipeline</B> est effectué à partir de trois type d'objets décrits ci-dessous :


 * <B>Transformer</B>: Un <B>Transformer</B> est un algorithme qui permet de transformer une DataFrame en une autre DataFrame. Dans la pluspart des cas, la nouvelle DataFrame est une DataFrame identique à la première avec une colonne supplémentaire. Exemple de <B>Transformer</B>: 
     * Un modèle d'apprentissage va prendre en entrée une DataFrame de variable et retourner une nouvelle DataFrame avec les variables et une nouvelle colonne correspondant à la prédiction.
     * Le Transformer <I>StringIndexer</I> va prendre en entrée une DataFrame possédant un colonne de texte et retourner une nouvelle DataFrame avec la même colonne texte et une nouvelle colonne ou les textes seront remplacés par une valeur numérique.
 

* <B>Estimator</B>: Un <B>Estimator</B> est un algorithme qui peut-être appliquer sur une DataFrame afin de produire un <B>Transformer</B>. Exemple d'<B>Estimator</B>:
    * Un algorithme d'apprentissage est un <B>Estimator</B>. Une fois celui-ci appliqué sur une DataFrame, il va produire un modèle d'apprentissage qui sera un <B>Transformer</B>, comme décrit précédemment.


 * <B>Parameter</B>: Chaque  <B>Transformer</B> et <B>Estimators</B> partage une même API pour spécifier leur paramètre.

#### Exemple : Regression Logistique

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# DataFrame D'apprentissage
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# On crée d'abord un objet LogisticRegression en spécifiant ses paramètres. Cet objet est un Estimator
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability')
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# On applique l'Estimator sur notre DataFrame d'apprentissage. L'objet qui en résulte, le modèle d'apprentissage, est un Transformer.
model = lr.fit(training)


# DataFrame de Test
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# On applique le Transformer sur la DataFrame de Test 'test'.
prediction = model.transform(test)

# Il en resulte une nouvelle DataFrame 'prediction' qui correspond a la DataFrame 'tes' auquelle on été ajoutés une colonne "prediction" et une colonne "probability".
# Les noms de ces nouvelles colonnes ont été spécifiés dans les paramètres lors de l'instanciation de l'Estimator 'lr' puis transmis au Transformer 'model'.

# Nous pouvons maintenant illustrer les résultats.
result = prediction.select("features", "label", "probability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.probability, row.prediction))

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction, current: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabili

### Pipeline

Un <B> Pipeline </B> est un enchainement de plusieurs <B> Transformers </B> et <B> Estimators </B> afin de spécifier un processus entier de Machine Learning. Par exemple, pour effectuer de l'apprentissage statistique sur des données textuelles, ces différentes étapes sont appliquées les unes à la suite des autres: 

 * Découpage du texte en liste de mot
 * Conversion en variable numérique
 * Aprentissahe sur les données numérique
 * Prédiction 

Toutes ces étapes peuvent être résumé dans un seul objet appelé <B> Pipeline </B>.

#### Exemple : Toeknize, Hash et Regression logistique

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# DataFrame d'Apprentissage
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure un pipeline qui consiste en 3 étapes :

# 1/ Tokenizer. On spécifie la colonne d'entrée 'text' et la colonne de sortie 'words'.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# 2/ Hash. La colonne d'entrée est spécifié ici comme étant la colonne de sortie de l'étape précédente.
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
# 3/ Regression Logistique
lr = LogisticRegression(maxIter=10, regParam=0.001)

# On configure le Pipeline comme la succession des étapes précédentes.
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# On applique toutes les étapes sur la DataFrame d'apprentissage.
model = pipeline.fit(training)

# DataFrame Test.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Prediction.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.155543713844,0.844456286156], prediction=1.000000
(5, l m n) --> prob=[0.830707735211,0.169292264789], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.0696218406195,0.93037815938], prediction=1.000000
(7, apache hadoop) --> prob=[0.981518350351,0.018481649649], prediction=0.000000


DataFrame[id: bigint, text: string]